In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import pymorphy3
import math
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [2]:
#nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/artemborisov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
stopWords = stopwords.words('russian')
stopWords.append('он')
stopWords.append('она')
stopWords.append('это')

morph = pymorphy3.MorphAnalyzer(lang='ru')

In [4]:
def delete_stop_words(text):
    return [word for word in text if word not in stopWords]

def format_data(text):
    fmtTextStr = re.sub(r'[^\w\s]', ' ', text.lower(), flags=re.UNICODE)
    tokenTextArr = nltk.word_tokenize(fmtTextStr)
    cleanTextArr = delete_stop_words(tokenTextArr)
    return " ".join(cleanTextArr)

def get_normalFormStr(line):
    return ' '.join([morph.parse(morph.parse(word)[0].normal_form for word in line.split() if morph.parse(word)[0].normal_form not in stopWords)])

In [5]:
def read_files(filename, format):
    i = 1
    dataArr = []
    while True:
        try:
            fd = open(filename + str(i) + format, encoding='utf-8')
        except Exception:
            break
        dataArr.append([i, fd.read()])

        fd.close()
        i += 1
    return dataArr

In [6]:
gameArr = read_files("./docs/1-game_", ".txt")
vlogArr = read_files("./docs/2-vlog_", ".txt")
techArr = read_files("./docs/3-tech_", ".txt")
reactArr = read_files("./docs/4-react_", ".txt")

In [7]:
gameDF = pd.DataFrame(gameArr, columns=['num_doc', 'text'])
gameDF["class"] = "game"

vlogDF = pd.DataFrame(vlogArr, columns=['num_doc', 'text'])
vlogDF["class"] = "vlog"

techDF = pd.DataFrame(techArr, columns=['num_doc', 'text'])
techDF["class"] = "tech"

reactDF = pd.DataFrame(reactArr, columns=['num_doc', 'text'])
reactDF["class"] = "react"

In [8]:
videosDF = pd.concat([gameDF, vlogDF, techDF, reactDF])
videosDF.head(10)

,num_doc,text,class
0,1,Этот ролик просто поражает своей графикой и ат...,game
1,2,Впечатляющее видео! Я был поражен деталями в б...,game
2,3,Эта игра точно обещает стать новым эталоном в ...,game
3,4,"Смотря на этот ролик, я просто не могу поверит...",game
4,5,"Меня порадовало то, как игра балансирует между...",game
5,6,Один из самых впечатляющих аспектов этого роли...,game
6,7,"Невероятно, как много деталей и внимания к мир...",game
7,8,"Мне очень нравится то, как разнообразны возмож...",game
8,9,Очень впечатляющий ролик! Не могу не отметить ...,game
9,10,Я просто ошеломлен масштабом и красотой этого ...,game


In [11]:
videosDF["formated"] = videosDF["text"].apply(lambda textFTale: get_normalFormStr(format_data(textFTale)))
videosDF["formated_withoutMorph"] = videosDF["text"].apply(lambda textFTale: format_data(textFTale))

AttributeError: 'generator' object has no attribute 'lower'